In [606]:
from os import path, getcwd
from IPython.display import Markdown as md
import pandas as pd
import numpy as np
import sqlalchemy as sa
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import patsy

strYearDBPath=path.join('..','DB', 'year.sqlite3') # путь к рабочей базе данных фактических рядов SQLite 
strExYearDBPath=path.join('..','DB', 'exog_year.sqlite3') # путь к рабочей базе данных экзогенных переменных SQLite
strExParamDBPath=path.join('..','DB', 'exog_param.sqlite3') # путь к рабочей базе данных экзогенных параметров SQLite
strSvodDBPath=path.join('..','DB', 'svod.sqlite3') # путь к рабочей базе данных итоговых рядов SQLite 

# константы базы данных SQLite3
strDatas_table='datas' # название таблицы с данными в базе данных SQLite
strHeaders_table='headers' # название таблицы с текстовыми и кодовыми именами рядов в базе данных SQLite


In [607]:
pd.set_option('display.max_columns',None)

In [608]:
# выбрать переменные по кодам, которые должны использоваться в текущем блоке

# общие для всех блоков фактические ряды
list_Year=('GDP_x','GDP_Iq','CPIAv','KeyRate','Unmpl_s','Inc_x','DispInc_I','ZPN_I',
           'ZPR_I','Invest_Iq','Retail_Iq','Activ_x','ER','Urals','Pop_x','HHAv')
# фактические данные для зависимых переменных из блока "ЖИЛИЩНОЕ СТРОИТЕЛЬСТВО"
list_YearZS=('BldProcMKD','VvodyMKDprivate','ProjectsPrivate_x','VvodyIZD','VvodySocFB',
             'VvodySocRB','AvSqVv','Vvodyunits','VvodyIZDunits','ZhilFond','ZhilFondIZD',
             'ZhilFondunits','ZhilFondIZDunits')
# общие для всех блоков экзогенные переменные
list_ExYear=('GDP_x','GDP_Iq','CPIAv','KeyRate','Unmpl_s','Inc_x','DispInc_I','ZPN_I',
             'ZPR_I','Invest_Iq','Retail_Iq','Activ_x','ER','Urals','Pop_x')
# общие для всех блоков экзогенные параметры
list_ExParam=('p_ProbDef','p_LabProdOnCost','p_GovPolOnCost','ConstrPriceIZD_x',
              'p_MortgLifeAv_x','HHAv')


# дополнительные данные "Жилищное строительство":
# фактические
list_ZSDop=('Soldaty_x','Badzhilfond_x','queue_x','zhiloutgor_x','zhiloutselo_x')

# экзогенные переменные (Пока выбытия берутся из EXOG, list_ExYearDop совпадает с list_YearDop)
# list_ZSExDop=('Soldaty_x','Badzhilfond_x','queue_x') # в дальнейшем будет использоваться эта строка

# экзогенные параметры
list_ZSExParamDop=('p_VvodyAll_x','p_VvodyMKD_x','p_sbernas_s','p_sq1room_x','p_sq3room_x',
                   'p_monvmkd_s','p_arendavmon_s','roa_s','yieldofz_s','premium_s',
                   'yieldarenda_s','nowplot_x','goalplot_x','goalplotprom_x',
                   'goalplotnew_x','shareprom_s','potenprom_x','plotprom_i')


# дополнительные данные со Свода (пока только для блока "Жилищное строительство"):
list_ZSSvod=('seb1m_x','price1mddy_x','price1mperv_x')

In [610]:
# общие
strSELECT_Year='''select datas.date, datas.value, headers.code2
                  from {datas_table} join {header_table}
                  on {datas_table}.code={header_table}.code
                  where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_Year)
strSELECT_YearZS='''select datas.date,datas.value,headers.code2 
                    from {datas_table} join {header_table} 
                    on {datas_table}.code={header_table}.code 
                    where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_YearZS)
strSELECT_ExYear='''select datas.date, datas.value, headers.code2 
                    from {datas_table} join {header_table} 
                    on {datas_table}.code={header_table}.code
                    where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ExYear)
strSELECT_ExParam='''select datas.date,datas.value,headers.code2 
                     from {datas_table} join {header_table} 
                     on {datas_table}.code={header_table}.code 
                     where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ExParam)

# для ЖС
strSELECT_ZSDop='''select datas.date, datas.value, headers.code2
                     from {datas_table} join {header_table}
                     on {datas_table}.code={header_table}.code
                     where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ZSDop)
strSELECT_ZSExParamDop='''select datas.date,datas.value,headers.code2 
                        from {datas_table} join {header_table} 
                        on {datas_table}.code={header_table}.code 
                        where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ZSExParamDop)


# из Свод для ЖС
strSELECT_ZSSvod='''select datas.date, datas.value, headers.code2
                     from {datas_table} join {header_table}
                     on {datas_table}.code={header_table}.code
                     where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ZSSvod)


conWorkDB=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strYearDBPath)) # connection к рабочей базе данных фактических рядов 
conWorkEx=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strExYearDBPath)) # connection к рабочей базе данных экзогенных переменных
conWorkExParam=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strExParamDBPath)) # connection к рабочей базе данных экзогенных параметров
conWorkSvod=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strSvodDBPath)) # connection к рабочей базе данных СВОД

pdfYear=pd.read_sql(strSELECT_Year, con=conWorkDB)
pdfYearZS=pd.read_sql(strSELECT_YearZS, con=conWorkDB)
pdfExYear=pd.read_sql(strSELECT_ExYear, con=conWorkEx)
pdfExParam=pd.read_sql(strSELECT_ExParam, con=conWorkExParam)

pdfZSDop=pd.read_sql(strSELECT_ZSDop,con=conWorkDB)
pdfZSExDop=pd.read_sql(strSELECT_ZSDop,con=conWorkEx)
pdfZSExParamDop=pd.read_sql(strSELECT_ZSExParamDop,con=conWorkExParam)

pdfZSSvodIN=pd.read_sql(strSELECT_ZSSvod,con=conWorkSvod)

In [584]:
idx=list(range(pdfYear['date'].min(),pdfYear['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfYearMain=pd.DataFrame(index=idx)
for i in pdfYear['code2'].unique():
    pdfYearMain[i]=pdfYear[pdfYear['code2']==i].set_index('date').drop(columns=['code2'])

pdfYearMain[['GDP_x','Inc_x']]*=1000
pdfYearMain[['GDP_Iq','CPIAv','KeyRate','Unmpl_s','DispInc_I','ZPN_I','ZPR_I','Invest_Iq','Retail_Iq']]*=100
pdfYearMain['Activ_x']/=1000
# pdfYearMain

In [556]:
idx=list(range(pdfYearZS['date'].min(),pdfYearZS['date'].max()+1)) # задается период от первой до последней даты в БД (2004 - 2019 гг.)
pdfYearZSMain=pd.DataFrame(index=idx)
for i in pdfYearZS['code2'].unique():
    pdfYearZSMain[i]=pdfYearZS[pdfYearZS['code2']==i].set_index('date').drop(columns=['code2'])
# pdfYearZSMain

In [557]:
idx=list(range(pdfExYear['date'].min(),pdfExYear['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfExYearMain=pd.DataFrame(index=idx)
for i in pdfExYear['code2'].unique():
    pdfExYearMain[i]=pdfExYear[pdfExYear['code2']==i].set_index('date').drop(columns=['code2'])
pdfExYearMain[['GDP_Iq','CPIAv','Unmpl_s','DispInc_I','ZPN_I','ZPR_I','Invest_Iq','Retail_Iq']]*=100
# pdfExYearMain

In [558]:
idx=list(range(pdfExParam['date'].min(),pdfExParam['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfExParamMain=pd.DataFrame(index=idx)
for i in pdfExParam['code2'].unique():
    pdfExParamMain[i]=pdfExParam[pdfExParam['code2']==i].set_index('date').drop(columns=['code2'])
# pdfExParamMain

In [559]:
pdfW=pdfYearMain.combine_first(pdfExYearMain).combine_first(pdfExParamMain)
# pdfW

In [651]:
pdfYearZSMain['IZDSqAv_x']=pdfYearZSMain['VvodyIZD']/pdfYearZSMain['VvodyIZDunits']
# pdfYearZSMain

#### Дальше данные с листа "Жилищное строительство"

In [579]:
pdfZSList=pdfW[['GDP_Iq','CPIAv','Unmpl_s','DispInc_I','Pop_x','HHAv']].rename(columns={'GDP_Iq':'GDP_rIq'})
pdfZSList[['CPIAv','Unmpl_s','DispInc_I']]/=100
pdfZSList['GDP_rIq']-=100
# pdfZSList

In [598]:
idx=list(range(pdfZSDop['date'].min(),pdfZSDop['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfZSDopM=pd.DataFrame(index=idx)
for i in pdfZSDop['code2'].unique():
    pdfZSDopM[i]=pdfZSDop[pdfZSDop['code2']==i].set_index('date').drop(columns=['code2'])
pdfZSDopM['Soldaty_x']*=1000000
pdfZSDopM['queue_x']*=1000

pdfZSDopM.loc[2019,['zhiloutgor_x','zhiloutselo_x']]=np.nan
# pdfZSDopM

In [599]:
idx=list(range(pdfZSExDop['date'].min(),pdfZSExDop['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfZSExDopM=pd.DataFrame(index=idx)
for i in pdfZSExDop['code2'].unique():
    pdfZSExDopM[i]=pdfZSExDop[pdfZSExDop['code2']==i].set_index('date').drop(columns=['code2'])
pdfZSExDopM['Soldaty_x']*=1000000
# pdfZSExDopM

In [602]:
idx=list(range(pdfZSExParamDop['date'].min(),pdfZSExParamDop['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfZSExParamDopM=pd.DataFrame(index=idx)
for i in pdfZSExParamDop['code2'].unique():
    pdfZSExParamDopM[i]=pdfZSExParamDop[pdfZSExParamDop['code2']==i].set_index('date').drop(columns=['code2'])
# pdfZSExParamDopM

In [614]:
idx=list(range(pdfZSSvodIN['date'].min(),pdfZSSvodIN['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfZSSvodINM=pd.DataFrame(index=idx)
for i in pdfZSSvodIN['code2'].unique():
    pdfZSSvodINM[i]=pdfZSSvodIN[pdfZSSvodIN['code2']==i].set_index('date').drop(columns=['code2'])
# pdfZSSvodINM

In [617]:
pdfWDop=pdfZSList.combine_first(pdfZSDopM).combine_first(pdfZSExDopM).combine_first(pdfZSExParamDopM).combine_first(pdfZSSvodINM)
# pdfWDop

### Модель 1

In [663]:
# Факторы
dfXX=pdfWDop[['seb1m_x','price1mddy_x','GDP_rIq']].copy()
dfXX['Marge']=dfXX['price1mddy_x']/dfXX['seb1m_x']*100
dfXX['Marge_1']=dfXX['Marge'].shift(1)
dfXX['GDP_rIq_1']=dfXX['GDP_rIq'].shift(1)
for i in dfXX.index.values[2:]:
    dfXX.loc[i,'Marge_rI']=dfXX.loc[i,'price1mddy_x']/dfXX.loc[i,'seb1m_x']*100-dfXX.loc[i-2,'price1mddy_x']/dfXX.loc[i-2,'seb1m_x']*100
dfXX['D_NewFinanialModel']=0
dfXX.loc[2019:2021,'D_NewFinanialModel']=[1,1,0.5]

# тренд
for i in pdfYearZSMain.index.values[2:]:
    dfXX.loc[i,'TrendF']=np.mean([pdfYearZSMain.loc[i-2,'BldProcMKD'],pdfYearZSMain.loc[i-1,'BldProcMKD']])
    dfXX['div_from_Trend']=pdfYearZSMain['BldProcMKD']-dfXX['TrendF']

dfXX

,seb1m_x,price1mddy_x,GDP_rIq,Marge,Marge_1,GDP_rIq_1,Marge_rI,D_NewFinanialModel,TrendF,div_from_Trend
2004,NaN,NaN,7.175949,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2005,13.812000,21.156722,6.376187,153.176381,NaN,7.175949,NaN,0.0,NaN,NaN
2006,16.840000,28.011468,8.153432,166.338886,153.176381,6.376187,NaN,0.0,63.125021,7.733918
2007,20.720000,39.056041,8.535080,188.494408,166.338886,8.153432,35.318027,0.0,68.861149,0.080825
2008,26.622000,45.686359,5.247954,171.611295,188.494408,8.535080,5.272409,0.0,69.900456,-5.227953
2009,30.312000,43.490170,-7.820885,143.475094,171.611295,5.247954,-45.019314,0.0,66.807239,1.669347
2010,31.877000,42.829263,4.503726,134.357885,143.475094,-7.820885,-37.253409,0.0,66.574545,7.263146
2011,33.320000,37.897473,4.264177,113.737913,134.357885,4.503726,-29.737180,0.0,71.157139,9.941505
2012,34.354000,41.598687,4.024086,121.088335,113.737913,4.264177,-13.269550,0.0,77.468168,12.850766
2013,36.439000,44.120233,1.755422,121.079702,121.088335,4.024086,7.341789,0.0,85.708789,14.278120


In [653]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [671]:
df=dfXX.loc[2011:2019,['div_from_Trend','Marge','Marge_rI','GDP_rIq_1','D_NewFinanialModel']]
# df

In [670]:
mod = smf.ols(formula='div_from_Trend ~ Marge.shift(1) + Marge_rI + GDP_rIq_1 + D_NewFinanialModel -1', data=dfXX.loc[2010:2019])
res = mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:         div_from_Trend   R-squared (uncentered):                   0.964
Model:                            OLS   Adj. R-squared (uncentered):              0.936
Method:                 Least Squares   F-statistic:                              33.65
Date:                Tue, 22 Dec 2020   Prob (F-statistic):                    0.000828
Time:                        20:20:15   Log-Likelihood:                         -19.222
No. Observations:                   9   AIC:                                      46.44
Df Residuals:                       5   BIC:                                      47.23
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

/home/egor/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [675]:
res.predict(dfXX)

2004          NaN
2005          NaN
2006          NaN
2007    35.016120
2008    29.368003
2009     8.725413
2010   -18.657591
2011     9.070768
2012    11.660029
2013    16.755677
2014     8.481863
2015     6.752919
2016     2.324905
2017     7.376764
2018    13.659970
2019   -12.384336
2020    -9.165210
2021     2.346419
2022    22.484519
2023    19.704935
2024    17.327466
2025    14.554919
2026    15.406472
2027    12.650437
2028    13.557513
2029    11.635343
2030    11.407368
2031          NaN
2032          NaN
2033          NaN
2034          NaN
2035          NaN
dtype: float64

## Дальше старые куски кода

Загрузка данных листа "Параметры" (27:26)

In [211]:
ddf=pd.read_excel(r'../DB/SRC/Модель развития ЖС_краткая версия_ИТОГ_Целевой_с новой моделью - новая доля продаж.xlsx',sheet_name=0,header=5,index_col=0)
ddf=ddf.loc[ddf.index.dropna()]
ddf=ddf.T

Загрузка данных листа "Жилищное строительство" (36:27)

In [206]:
hddf=pd.read_excel(r'../DB/SRC/Модель развития ЖС_краткая версия_ИТОГ_Целевой_с новой моделью - новая доля продаж.xlsx',sheet_name=3,header=5,index_col=0,nrows=45)
hddf=hddf.loc[hddf.index.dropna()]
hddf=hddf.iloc[:,1:-2]
#hddf.drop(hddf.columns[0],axis=1,inplace=True)
hddf=hddf.T

<div style="background:beige"><h3> Определение основных дат</h3>

  - `iLastFactYEAR` - последний год фактических данных. 2019
  - `iLastFORCAST` - год горизонта прогноза. 2030

    Пока задаются вручную
</div>

In [207]:
# iLastFactYEAR
iLastFactYEAR=conWork.connect().execute('select max(date) from datas').fetchone()[0]
iLastFORCAST=iLastFactYEAR+11

print('Последний фактический год - ', iLastFactYEAR, '; горизонт прогноза - ', iLastFORCAST)

Последний фактический год -  2019 ; горизонт прогноза -  2030


In [208]:
hddf.index=hddf.index.astype(int)
hddf_XY=hddf.truncate(after=iLastFactYEAR)
hddf_xy=hddf.truncate(before=iLastFactYEAR+1, after=iLastFORCAST)

In [210]:
hddf_XY

,Макроэкономические и демографические факторы,Темп прироста физического объема ВВП,"ИПЦ, в среднем за год",Уровень безработицы,Изменение реальных располагаемых доходов населения,"Численность населения, в среднем за год",Средний размер домохозяйства,Параметры данного блока,"Совокупный ввод жилья в рамках Национального проекта ""Жилье и городская среда""","Совокупный ввод жилья в МКД в рамках Национального проекта ""Жилье и городская среда""",...,Целевая плотность застройки в двадцати крупных городских агломерациях,Целевая плотность застройки в промышленных зонах городских центров двадцати крупных городских агломераций,Целевая плотность застройки на освоенных новых территориях в двадцати крупных городских агломерациях,Доля площади застраиваемых промышленных зон в черте двадцати центров крупных городских агломераций,Потенциал строительства жилья во всех промышленных зонах в черте двадцати центров крупных городских агломерациях,Коэффициент повышения плотности строительства в промышленных зонах,Параметры из других блоков модели,Средняя фактическая удельная стоимость строительства МКД,"Средняя цена 1 кв.м, приобретаемого по ДДУ, в МКД",Средняя цена 1 кв. м на первичном рынке
2005,NaN,6.37619,1.12706,0.0715324,1.124,143.519,2.66147,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.812,21.1567,23.7716
2006,NaN,8.15343,1.09683,0.0706222,1.135,143.05,2.64589,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.84,28.0115,31.4736
2007,NaN,8.53508,1.08988,0.0602939,1.121,142.805,2.6303,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.72,39.056,43.8832
2008,NaN,5.25646,1.14105,0.0624119,1.024,142.742,2.61472,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.622,45.6864,51.333
2009,NaN,-7.82474,1.11698,0.0825224,1.03,142.785,2.59913,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.312,43.4902,48.8654
2010,NaN,18.3923,1.06851,0.0734454,1.059,142.849,2.58355,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.877,42.8293,48.1228
2011,NaN,0.467491,1.08463,0.0649564,1.005,142.961,2.54416,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.32,37.8975,42.5814
2012,NaN,4.0176,1.05067,0.0545866,1.046,143.202,2.50476,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.354,41.5987,46.7401
2013,NaN,1.75175,1.06761,0.0547804,1.04,143.507,2.46537,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.439,44.1202,49.5733
2014,NaN,0.744972,1.07804,0.0515632,0.988,146.091,2.42598,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.447,45.3205,50.9219


## Запас Жилья строящихся МКД

In [243]:
pdfNvstr['Тренд'] = None

In [256]:
pdfNvstr

,value,Тренд
date,,
2004,59.386684,63.125021
2005,66.863359,63.125021
2006,70.858939,68.861149
2007,68.941974,69.900456
2008,64.672504,66.807239
2009,68.476586,66.574545
2010,73.837691,71.157139
2011,81.098644,77.468168
2012,90.318934,85.708789


In [255]:
for i, row in pdfNvstr.iterrows():
    print(row.name)
    pdfNvstr['Тренд'][row.name+1]=np.mean([pdfNvstr['value'][row.name],pdfNvstr['value'][row.name+1]])

2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


KeyError: 2020